In [1]:
import os
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from mlxtend.classifier import StackingCVClassifier
import joblib
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
import pandas as pd

random_seed = 1234

KFOLD_SPLIT = 5

# --- X_smote + 10 fet using rf-rfe ---
# 1234， 2345 is split 3
# 2345, 42 is split 5
# 333 : split 10

# --- X_ros + 10 fet using rf-rfe ---
# 111， split 3
# 222, split 5
# 555, split 10

# --- X_smote + 48 fet ---
# 25, split 3
# 35, split 5
# 65, split 10

# --- X_ros + 48 fet ---
# 255, split 3
# 355, split 5
# 655, split 10

# --- X_ros + 14 fet using be ---
# 616: split 3
# 626: split 5
# 636: split 10

# --- X_smote + 14 fet using be ---
# 716: split 3
# 726: split 5
# 736: split 10

# --- 10 fet using be sfs backward ---
# 666: split 3

# --- 10 fet using be sfs forward ---
# 888: split 3

DATA_DIR = '/Users/malithidesilva/fyp/model_1/final'
RESULT_DIR = f'/Users/malithidesilva/fyp/model_1/final/model1_results_{random_seed}'



os.makedirs(RESULT_DIR, exist_ok=True)

# Define models
rf_model = RandomForestClassifier(random_state=random_seed)
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state = random_seed)
dt_model = DecisionTreeClassifier(random_state=random_seed)
gbm_model = GradientBoostingClassifier(n_estimators=100, random_state=random_seed)
svc_model = SVC(probability=True, random_state=random_seed)
knn_model = KNeighborsClassifier(n_neighbors=5)



single_pipelines = {
    'RF': ImbPipeline([
        ('scaling', MinMaxScaler()),
        ('classifier', rf_model)
    ]),
    'XGBoost': ImbPipeline([
        ('scaling', MinMaxScaler()),
        ('classifier', xgb_model)
    ]),
    'DT': ImbPipeline([
        ('scaling', MinMaxScaler()),
        ('classifier', dt_model)
    ]),
    'GBM': ImbPipeline([
        ('scaling', MinMaxScaler()),
        ('classifier', gbm_model)
    ]),
    'SVC': ImbPipeline([
        ('scaling', MinMaxScaler()),
        ('classifier', svc_model)
    ]),
    'KNN': ImbPipeline([
        ('scaling', MinMaxScaler()),
        ('classifier', knn_model)
    ]),
    
}

ensemble_pipelines = {
    'ALL_RF': ImbPipeline([
        ('scaling', MinMaxScaler()),
        ('classifier', StackingCVClassifier(
            classifiers=[rf_model, xgb_model, dt_model, gbm_model, svc_model, knn_model],
            meta_classifier=rf_model,
            cv=KFOLD_SPLIT,
        ))
    ]),
    'ALL_XGBoost': ImbPipeline([
        ('scaling', MinMaxScaler()),
        ('classifier', StackingCVClassifier(
            classifiers=[rf_model, xgb_model, dt_model, gbm_model, svc_model, knn_model],
            meta_classifier=xgb_model,
            cv=KFOLD_SPLIT,
        ))
    ]),
    'ALL_SVC': ImbPipeline([
        ('scaling', MinMaxScaler()),
        ('classifier', StackingCVClassifier(
            classifiers=[rf_model, xgb_model, dt_model, gbm_model, svc_model, knn_model],
            meta_classifier=svc_model,
            cv=KFOLD_SPLIT,
        ))
    ]),
    'ALL_GBM': ImbPipeline([
        ('scaling', MinMaxScaler()),
        ('classifier', StackingCVClassifier(
            classifiers=[rf_model, xgb_model, dt_model, gbm_model, svc_model, knn_model],
            meta_classifier=gbm_model,
            cv=KFOLD_SPLIT,
        ))
    ])
}



In [2]:
def extended_class_report(cm):
    tn, fp, fn, tp = cm.ravel()

    # Calculate additional metrics
    ppv = tp / (tp + fp) if (tp + fp) != 0 else 0
    npv = tn / (tn + fn) if (tn + fn) != 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0

    return ppv, npv, sensitivity, specificity

from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

def draw_auc(actual_y, prob_y, saved_dir):

    y = actual_y #label_binarize(actual_y, classes=np.unique(actual_y))
    n_classes = y.shape[1]

    plt.figure(figsize=(15,15))
    # Compute ROC AUC for each class
    for i in range(n_classes):
        # Ensure prob_y is a numpy array, handle if it's a dataframe
        if hasattr(prob_y, "to_numpy"):
            prob_y = prob_y.to_numpy()
        fpr, tpr, threshold = roc_curve(y[:, i], prob_y[:, i])
        roc_auc_results = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f'Class {i} (area = {roc_auc_results:.2f})')

    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Multiclass ROC Curve by One-vs-Rest, thres:{threshold}')
    plt.legend(loc="lower right")
    # plt.show()
    plt.savefig(saved_dir)
    plt.close()

    # Function to convert numpy array to DataFrame
def ensure_dataframe(data):
    if isinstance(data, np.ndarray):
        return pd.DataFrame(data)
    return data

In [3]:
def evaluate_models(pipelines, X_train_selected, y_train_selected, X_test_selected, y_test_selected, result_suffix):
    overview = []
    for model_name, pipeline in pipelines.items():
        model_result_dir = os.path.join(RESULT_DIR, f"{model_name}_{result_suffix}")
        if not os.path.exists(model_result_dir):
            os.makedirs(model_result_dir)
        print(f'------- {model_name} -------')

        print('>>> Fitting training model')

        pipeline.fit(X_train_selected, y_train_selected)
        y_train_pred = cross_val_predict(pipeline, X_train_selected, y_train_selected, cv=KFOLD_SPLIT)
        y_train_pred_proba = cross_val_predict(pipeline, X_train_selected, y_train_selected, cv=KFOLD_SPLIT, method='predict_proba')[:, 1]

        train_accuracy = accuracy_score(y_train_selected, y_train_pred)
        train_conf_matrix = confusion_matrix(y_train_selected, y_train_pred)
        train_pv, train_npv, train_sensitivity, train_specificity = extended_class_report(train_conf_matrix)
        train_roc_auc = roc_auc_score(y_train_selected, y_train_pred_proba)

        print('>>> Validating test set')

        y_test_pred = pipeline.predict(X_test_selected)
        y_test_pred_proba = pipeline.predict_proba(X_test_selected)[:, 1]

        test_accuracy = accuracy_score(y_test_selected, y_test_pred)
        test_conf_matrix = confusion_matrix(y_test_selected, y_test_pred)
        test_pv, test_npv, test_sensitivity, test_specificity = extended_class_report(test_conf_matrix)
        test_roc_auc = roc_auc_score(y_test_selected, y_test_pred_proba)

        print(f'Results: {train_roc_auc}, {test_roc_auc}')

        pd.DataFrame(train_conf_matrix).to_csv(os.path.join(model_result_dir, f'{model_name}_train_cm.csv'))
        pd.DataFrame(test_conf_matrix).to_csv(os.path.join(model_result_dir, f'{model_name}_test_cm.csv'))

        overview.append([model_name, test_accuracy, test_roc_auc, test_pv, test_npv, test_sensitivity, test_specificity, train_accuracy, train_roc_auc, train_pv, train_npv, train_sensitivity, train_specificity, train_roc_auc-test_roc_auc])

        pred_to_save = {
            'y_train_pred': y_train_pred,
            'y_train_pred_proba': y_train_pred_proba,
            'y_test_pred': y_test_pred,
            'y_test_pred_proba': y_test_pred_proba,
        }

        joblib.dump(pred_to_save, os.path.join(model_result_dir, f'{model_name}_pred_values.pkl'))
        joblib.dump(pipeline, os.path.join(model_result_dir, f'{model_name}_full_model.pkl'))

    pd.DataFrame(overview, columns=['model_name', 
                                    'test_accuracy', 
                                    'test_roc_auc', 
                                    'test_ppv', 
                                    'test_npv', 
                                    'test_sensitivity', 
                                    'test_specificity', 
                                    'train_accuracy', 
                                    'train_roc_auc', 
                                    'train_ppv', 
                                    'train_npv', 
                                    'train_sensitivity', 
                                    'train_specificity',
                                    'auc_diff'
                                    ]).to_csv(os.path.join(RESULT_DIR, f'results_overview_{result_suffix}.csv'))


In [4]:
from sklearn.model_selection import cross_val_score


def evaluate_ensemble_models(pipelines, X_train_selected, y_train_selected, X_test_selected, y_test_selected, result_suffix):
    overview = []
    for model_name, pipeline in pipelines.items():
        model_result_dir = os.path.join(RESULT_DIR, f"{model_name}_{result_suffix}")
        if not os.path.exists(model_result_dir):
            os.makedirs(model_result_dir)
        print(f'------- {model_name} -------')

        print('>>> Fitting training model')

        pipeline.fit(X_train_selected, y_train_selected)
        
        # Use cross_val_predict only for classification tasks where classes_ attribute exists
        if hasattr(pipeline, 'classes_'):
            y_train_pred = cross_val_predict(pipeline, X_train_selected, y_train_selected, cv=KFOLD_SPLIT)
            y_train_pred_proba = cross_val_predict(pipeline, X_train_selected, y_train_selected, cv=KFOLD_SPLIT, method='predict_proba')[:, 1]
        else:
            # Directly predict on training data
            y_train_pred = pipeline.predict(X_train_selected)
            y_train_pred_proba = pipeline.predict_proba(X_train_selected)[:, 1]

        train_accuracy = accuracy_score(y_train_selected, y_train_pred)
        train_conf_matrix = confusion_matrix(y_train_selected, y_train_pred)
        train_pv, train_npv, train_sensitivity, train_specificity = extended_class_report(train_conf_matrix)
        train_roc_auc = roc_auc_score(y_train_selected, y_train_pred_proba)

        print('>>> Validating test set')

        y_test_pred = pipeline.predict(X_test_selected)
        y_test_pred_proba = pipeline.predict_proba(X_test_selected)[:, 1]

        test_accuracy = accuracy_score(y_test_selected, y_test_pred)
        test_conf_matrix = confusion_matrix(y_test_selected, y_test_pred)
        test_pv, test_npv, test_sensitivity, test_specificity = extended_class_report(test_conf_matrix)
        test_roc_auc = roc_auc_score(y_test_selected, y_test_pred_proba)

        print(f'Results: {train_roc_auc}, {test_roc_auc}')

        pd.DataFrame(train_conf_matrix).to_csv(os.path.join(model_result_dir, f'{model_name}_train_cm.csv'))
        pd.DataFrame(test_conf_matrix).to_csv(os.path.join(model_result_dir, f'{model_name}_test_cm.csv'))

        overview.append([model_name, test_accuracy, test_roc_auc, test_pv, test_npv, test_sensitivity, test_specificity, train_accuracy, train_roc_auc, train_pv, train_npv, train_sensitivity, train_specificity, train_roc_auc-test_roc_auc])

        pred_to_save = {
            'y_train_pred': y_train_pred,
            'y_train_pred_proba': y_train_pred_proba,
            'y_test_pred': y_test_pred,
            'y_test_pred_proba': y_test_pred_proba,
        }

        joblib.dump(pred_to_save, os.path.join(model_result_dir, f'{model_name}_pred_values.pkl'))
        joblib.dump(pipeline, os.path.join(model_result_dir, f'{model_name}_full_model.pkl'))

    pd.DataFrame(overview, columns=['model_name', 
                                    'test_accuracy', 
                                    'test_roc_auc', 
                                    'test_ppv', 
                                    'test_npv', 
                                    'test_sensitivity', 
                                    'test_specificity', 
                                    'train_accuracy', 
                                    'train_roc_auc', 
                                    'train_ppv', 
                                    'train_npv', 
                                    'train_sensitivity', 
                                    'train_specificity',
                                    'auc_diff'
                                    ]).to_csv(os.path.join(RESULT_DIR, f'results_overview_{result_suffix}.csv'))


In [5]:
def evaluate_shap_models(pipelines, X_train_selected, y_train_selected, X_test_selected, y_test_selected, result_suffix, is_shap=False):

    overview = []
    for model_name, pipeline in pipelines.items():
        model_result_dir = os.path.join(RESULT_DIR, f"{model_name}_{result_suffix}")
        if not os.path.exists(model_result_dir):
            os.makedirs(model_result_dir)
        print(f'------- {model_name} evaluation {result_suffix} -------')

        print('>>> Fitting training model')

        pipeline.fit(X_train_selected, y_train_selected)
        y_train_pred = cross_val_predict(pipeline, X_train_selected, y_train_selected, cv=KFOLD_SPLIT)
        y_train_pred_proba = cross_val_predict(pipeline, X_train_selected, y_train_selected, cv=KFOLD_SPLIT, method='predict_proba')[:, 1]

        train_accuracy = accuracy_score(y_train_selected, y_train_pred)
        train_conf_matrix = confusion_matrix(y_train_selected, y_train_pred)
        train_pv, train_npv, train_sensitivity, train_specificity = extended_class_report(train_conf_matrix)
        train_roc_auc = roc_auc_score(y_train_selected, y_train_pred_proba)

        print('>>> Validating test set')

        y_test_pred = pipeline.predict(X_test_selected)
        y_test_pred_proba = pipeline.predict_proba(X_test_selected)[:, 1]

        test_accuracy = accuracy_score(y_test_selected, y_test_pred)
        test_conf_matrix = confusion_matrix(y_test_selected, y_test_pred)
        test_pv, test_npv, test_sensitivity, test_specificity = extended_class_report(test_conf_matrix)
        test_roc_auc = roc_auc_score(y_test_selected, y_test_pred_proba)

        print(f'Results: Train ROC AUC = {train_roc_auc}, Test ROC AUC = {test_roc_auc}')

        pd.DataFrame(train_conf_matrix).to_csv(os.path.join(model_result_dir, f'{model_name}_train_cm.csv'))
        pd.DataFrame(test_conf_matrix).to_csv(os.path.join(model_result_dir, f'{model_name}_test_cm.csv'))

        overview.append([model_name, test_accuracy, test_roc_auc, test_pv, test_npv, test_sensitivity, test_specificity, 
                         train_accuracy, train_roc_auc, train_pv, train_npv, train_sensitivity, train_specificity, 
                         train_roc_auc-test_roc_auc])

        pred_to_save = {
            'y_train_pred': y_train_pred,
            'y_train_pred_proba': y_train_pred_proba,
            'y_test_pred': y_test_pred,
            'y_test_pred_proba': y_test_pred_proba,
        }

        joblib.dump(pred_to_save, os.path.join(model_result_dir, f'{model_name}_pred_values.pkl'))
        joblib.dump(pipeline, os.path.join(model_result_dir, f'{model_name}_full_model.pkl'))

    pd.DataFrame(overview, columns=['model_name', 'test_accuracy', 'test_roc_auc', 'test_ppv', 'test_npv', 'test_sensitivity', 
                                    'test_specificity', 'train_accuracy', 'train_roc_auc', 'train_ppv', 'train_npv', 
                                    'train_sensitivity', 'train_specificity', 'auc_diff']).to_csv(os.path.join(RESULT_DIR, f'results_overview_{result_suffix}.csv'))






In [ ]:
def evaluate_shap_ensemble_models(pipelines, X_train_selected, y_train_selected, X_test_selected, y_test_selected, result_suffix, is_shap=False):
    overview = []
    for model_name, pipeline in pipelines.items():
        model_result_dir = os.path.join(RESULT_DIR, f"{model_name}_{result_suffix}")
        if not os.path.exists(model_result_dir):
            os.makedirs(model_result_dir)
        print(f'------- {model_name} evaluation {result_suffix} -------')

        print('>>> Fitting training model')

        pipeline.fit(X_train_selected, y_train_selected)
        
        # Use cross_val_predict only for classification tasks where classes_ attribute exists
        if hasattr(pipeline, 'classes_'):
            y_train_pred = cross_val_predict(pipeline, X_train_selected, y_train_selected, cv=KFOLD_SPLIT)
            y_train_pred_proba = cross_val_predict(pipeline, X_train_selected, y_train_selected, cv=KFOLD_SPLIT, method='predict_proba')[:, 1]
        else:
            # Directly predict on training data
            y_train_pred = pipeline.predict(X_train_selected)
            y_train_pred_proba = pipeline.predict_proba(X_train_selected)[:, 1]

        train_accuracy = accuracy_score(y_train_selected, y_train_pred)
        train_conf_matrix = confusion_matrix(y_train_selected, y_train_pred)
        train_pv, train_npv, train_sensitivity, train_specificity = extended_class_report(train_conf_matrix)
        train_roc_auc = roc_auc_score(y_train_selected, y_train_pred_proba)

        print('>>> Validating test set')

        y_test_pred = pipeline.predict(X_test_selected)
        y_test_pred_proba = pipeline.predict_proba(X_test_selected)[:, 1]

        test_accuracy = accuracy_score(y_test_selected, y_test_pred)
        test_conf_matrix = confusion_matrix(y_test_selected, y_test_pred)
        test_pv, test_npv, test_sensitivity, test_specificity = extended_class_report(test_conf_matrix)
        test_roc_auc = roc_auc_score(y_test_selected, y_test_pred_proba)

        print(f'Results: Train ROC AUC = {train_roc_auc}, Test ROC AUC = {test_roc_auc}')

        pd.DataFrame(train_conf_matrix).to_csv(os.path.join(model_result_dir, f'{model_name}_train_cm.csv'))
        pd.DataFrame(test_conf_matrix).to_csv(os.path.join(model_result_dir, f'{model_name}_test_cm.csv'))

        overview.append([model_name, test_accuracy, test_roc_auc, test_pv, test_npv, test_sensitivity, test_specificity, 
                         train_accuracy, train_roc_auc, train_pv, train_npv, train_sensitivity, train_specificity, 
                         train_roc_auc-test_roc_auc])

        pred_to_save = {
            'y_train_pred': y_train_pred,
            'y_train_pred_proba': y_train_pred_proba,
            'y_test_pred': y_test_pred,
            'y_test_pred_proba': y_test_pred_proba,
        }

        joblib.dump(pred_to_save, os.path.join(model_result_dir, f'{model_name}_pred_values.pkl'))
        joblib.dump(pipeline, os.path.join(model_result_dir, f'{model_name}_full_model.pkl'))

    pd.DataFrame(overview, columns=['model_name', 'test_accuracy', 'test_roc_auc', 'test_ppv', 'test_npv', 'test_sensitivity', 
                                    'test_specificity', 'train_accuracy', 'train_roc_auc', 'train_ppv', 'train_npv', 
                                    'train_sensitivity', 'train_specificity', 'auc_diff']).to_csv(os.path.join(RESULT_DIR, f'results_overview_{result_suffix}.csv'))

In [7]:

import pickle

# Load N values from the file

with open('N_values.pkl', 'rb') as f:
    N_values = pickle.load(f)

# Define dataset files with patterns
dataset_files = {
    'resampled_data': 'resampled_data_smote.pkl',
    'rfe_data': 'selected_fet_data_RFE_rf.pkl',
    'be_data': 'selected_fet_data_be.pkl',
    'reduced_data': 'reduced_data.pkl',
    'shap_data': 'shap_reduced_data_{N}.pkl'  # Pattern for SHAP data
}

# Main evaluation loop
for dataset_name, data_file in dataset_files.items():
    print(f"Loading and evaluating {dataset_name}...")

    if dataset_name == 'shap_data':
        for N in N_values:
            file_path = os.path.join(DATA_DIR, data_file.format(N=N))
            try:
                data = joblib.load(file_path)
                X_train_selected = data['X_train_shap']
                y_train_selected = data['y_train_selected']
                X_test_selected = data['X_test_shap']
                y_test_selected = data['y_test_selected']

                # Evaluate single pipelines
                evaluate_shap_models(single_pipelines, X_train_selected, y_train_selected, X_test_selected, y_test_selected, result_suffix=f'shap_data_N_{N}', is_shap=True)
                
                # Evaluate ensemble pipelines
                evaluate_shap_ensemble_models(ensemble_pipelines, X_train_selected, y_train_selected, X_test_selected, y_test_selected, result_suffix=f'ensemble_shap_data_N_{N}', is_shap=True)
                
                print(f"Completed evaluation for SHAP data with N={N}.")
            except FileNotFoundError:
                print(f"File not found: {file_path}")
    else:
        file_path = os.path.join(DATA_DIR, data_file)
        try:
            data = joblib.load(file_path)
            if dataset_name == 'resampled_data':
                X_train_selected = data['X_balanced']
                y_train_selected = data['y_balanced']
                X_test_selected = data['X_test_selected']
                y_test_selected = data['y_test']
            elif dataset_name == 'rfe_data':
                X_train_selected = data['X_train_rfe']
                y_train_selected = data['y_train_selected']
                X_test_selected = data['X_test_rfe']
                y_test_selected = data['y_test_selected']
            elif dataset_name == 'be_data':
                X_train_selected = data['X_train_be']
                y_train_selected = data['y_train_selected']
                X_test_selected = data['X_test_be']
                y_test_selected = data['y_test_selected']
            elif dataset_name == 'reduced_data':
                X_train_selected = data['X_train_reduced']
                y_train_selected = data['y_train_selected']
                X_test_selected = data['X_test_reduced']
                y_test_selected = data['y_test_selected']

            # Evaluate single pipelines
            evaluate_models(single_pipelines, X_train_selected, y_train_selected, X_test_selected, y_test_selected, result_suffix=dataset_name)
            
            # Evaluate ensemble pipelines
            evaluate_ensemble_models(ensemble_pipelines, X_train_selected, y_train_selected, X_test_selected, y_test_selected, result_suffix=f'ensemble_{dataset_name}')
            
            print(f"Completed evaluation for {dataset_name}.")
        except FileNotFoundError:
            print(f"File not found: {file_path}")

print("Evaluation completed for all datasets.")


Loading and evaluating resampled_data...
------- RF -------
>>> Fitting training model


>>> Validating test set
Results: 0.8939625, 0.8226009043711271
------- XGBoost -------
>>> Fitting training model
>>> Validating test set
Results: 0.882775, 0.8118126500307039
------- DT -------
>>> Fitting training model
>>> Validating test set
Results: 0.7474999999999999, 0.6241556411544688
------- GBM -------
>>> Fitting training model
>>> Validating test set
Results: 0.8556, 0.8271367163512533
------- SVC -------
>>> Fitting training model
>>> Validating test set
Results: 0.882675, 0.7868307932786245
------- KNN -------
>>> Fitting training model
>>> Validating test set
Results: 0.8180499999999999, 0.751256071009881
------- ALL_RF -------
>>> Fitting training model
>>> Validating test set
Results: 0.995, 0.787514654161782
------- ALL_XGBoost -------
>>> Fitting training model
>>> Validating test set
Results: 0.9999999999999999, 0.8278066208898565
------- ALL_SVC -------
>>> Fitting training model
>>> Validating test set
Results: 1.0, 0.7245575838776308
------- ALL_GBM -------
>>> F